In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires safetensors>=0.4.3, which is not installed.
transformers 4.57.3 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.


# 1. knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법with_table.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


194

In [2]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [3]:
len(embedding.embed_query("소득세법"))

4096

In [6]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-table"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
# database = PineconeVectorStore(
#     embedding=embedding,# 질문을 임베딩하여 유사도 검색
#     index_name=index_name
# )

CPU times: total: 7.16 s
Wall time: 1min 3s


# 2. 답변 생성을 위한 Retrieval

In [7]:
query = "연봉이 5천만 원 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [8]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [16]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [19]:
retrieved_docs[1]

Document(id='ccd7cb95-9158-416c-9d28-7cd526e9367e', metadata={'source': 'data/소득세법with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준 및 세율\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31

# 3. 답변 생성

In [21]:
import os
from openai import OpenAI
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [22]:
print(response.choices[0].message.content)

연봉 5천만 원 직장인의 근로소득세액공제는 다음과 같이 계산됩니다:  

1. **근로소득세액공제액**:  
   - 74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = **66만원**  
     (계산 결과 66만원 미만이므로 최소 66만원 적용)  

2. **종합소득산출세액**:  
   - 제55조 세율 적용 시 5,000만원 이하 구간에서 **624만원 + (5,000만원 - 1,400만원) × 15% = 624만원 + 540만원 = 1,164만원**  

3. **공제 후 세액**:  
   - 1,164만원 - 66만원 = **1,098만원**  

※ 단, 실제 납부세액은 추가 공제(자녀세액공제 등) 및 지방소득세(10%)가 반영될 수 있습니다.  

(답변: 10줄 이내 요약)


In [11]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [12]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [13]:
ai_message = llm.invoke(prompt)

In [14]:
print(ai_message.content)

소득세법 및 제공된 [context]를 기반으로 연봉 5천만원인 직장인의 소득세를 계산하는 방법은 다음과 같습니다. 다만, 정확한 계산을 위해서는 근로소득공제율과 세율표 등 구체적인 수치가 필요하나, [context]에 명시된 내용을 중심으로 일반적인 절차를 설명드립니다.

---

### **1. 근로소득공제 적용 (제47조)**
- **근로소득공제**는 총급여액에서 일정 금액을 공제하는 제도입니다.  
- 공제율은 총급여액 구간에 따라 차등 적용되며, **최대 2천만원**까지 공제 가능합니다.  
- 예시 (실제 계산 시 세법 별표 참고 필요):  
  - 총급여액 5천만원일 경우, 근로소득공제액은 약 **1,200만원 ~ 1,500만원** (가칭, 정확한 수치는 세법 별표 기준)으로 추정됩니다.  
- **과세표준** = 총급여액(5,000만원) - 근로소득공제액(예: 1,500만원) = **3,500만원**

---

### **2. 종합소득산출세액 계산 (세율 적용)**
- 과세표준에 **종합소득세율**을 적용합니다 (2024년 기준 누진세율):  
  - 1,200만원 이하: 6%  
  - 1,200만원 ~ 4,600만원: 15%  
  - 4,600만원 ~ 8,800만원: 24%  
  - ... (고액 구간 생략)  
- 예시 계산:  
  - 1,200만원 × 6% = **72만원**  
  - (4,600만원 - 1,200만원) × 15% = **510만원**  
  - (3,500만원 - 4,600만원) → **해당 없음** (과세표준이 3,500만원이므로)  
  - **종합소득산출세액** = 72만원 + 510만원 = **582만원**

---

### **3. 근로소득세액공제 적용 (제59조)**
- 근로소득세액공제는 산출세액에서 추가 공제됩니다.  
- 총급여액 5,000만원인 경우:  
  - 공제액 = **74만원 - (5,000만원 - 3,300만원) × 0.08%**  
    = 74만원 - 1,400만원 × 0.0008 

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [16]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [17]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'제공된 문맥에는 연봉 5천만원에 대한 구체적인 소득세 계산 방법이 명시되어 있지 않습니다. 따라서 정확한 세액을 계산할 수 없으며, "모름"으로 답변드립니다. 단, 근로소득 공제 및 세액 공제 규정을 참고하여 국세청의 세금 계산기 등을 활용하면 예상 세액을 확인할 수 있습니다.'